In [1]:
using DataFrames, DataFramesMeta, Revise, CSV, Serialization

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;


┌ Info: Precompiling BLPmodule [top-level]
└ @ Base loading.jl:1317


In [3]:
const datadir = "/export/storage_adgandhi/MiscLi/factract";

# test mode
test1state = false
if test1state 
    data = DataFrame(CSV.File("$datadir/analysis/factract_FL.csv"));
    dirpath = "/export/storage_adgandhi/MiscLi/factract/analysis"
else
    data = DataFrame(CSV.File("$datadir/analysis/factract.csv"));
    dirpath = "jls"
end;

In [4]:
println(names(data))

["facid", "year", "county", "occpct", "totbeds", "dchrppd", "rnhrppd", "avg_dailycensus", "lat", "lon", "restot", "state", "cz", "tractid", "dist", "pop65plus_int", "poptot_int", "pop65_j", "fracpop_inmkt", "mktpop", "nres_mcare", "nres_mcaid", "nbr_dchrppd", "nbr_rnhrppd", "tractyear"]


In [5]:
df = @select(data, 
    :t = :tractyear, 
    :tract = :tractid, 
    :year = :year, 
    :state = :state, 
    :j = :facid, 
    :q = :restot,
    :avg_dailycensus,
    :nres_mcare,
    :nres_mcaid,
    :M = :mktpop,
    :d = :dist, 
    :d2 = :dist .^ 2,
    :x1 = :dchrppd,
    :x2 = :rnhrppd,
    :z1 = :nbr_dchrppd,
    :z2 = :nbr_rnhrppd,
);



In [ ]:
@subset!(df, :state.=="FL", :year.>2007);

In [ ]:

firm_IDs_long = df.j;
tract_IDs_long = df.t;
X = Matrix(df[!, [:x1, :x2]])
Z = Matrix(df[!, [:z1, :z2]])
D = Matrix(df[!, [:d, :d2]])
Q = df.q
M = df.M;
nJ = length(unique(firm_IDs_long));

# economy for NLLS 
ec = m.make_Economy(
    firm_IDs_long,
    tract_IDs_long,
    X, Z, D, Q, M, 1
    );



In [ ]:
q_obs = ec.q_obs;
tracts = ec.tracts;
n_firms = length(ec.firms);
X = [t.X[:,1:2] for t in tracts];
D = [t.D[:,1:2] for t in tracts];
j_indexer, t_indexer, positionin = m.create_indices(tracts, n_firms);


In [ ]:
θx, θd, share, mktq, firmq = m.run_nlls(X, D, tracts, n_firms, q_obs);